In [ ]:
pip install plotly 

In [ ]:
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import warnings
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
BOOT_NUM = 100


##Data Preprocess:

In [ ]:
students_columns = ['gender','race',
                    'g1tmathss','yearssmall',
                    'g3thighdegree', 'g3tyears', 'g3freelunch',
                    'g3absent', 'g1speced', 'g3classsize','g3schid','g3tmathss','g3classtype']

schools_columns = ['schid','SCHLURBN', 'G3AVGDAT', 'G3ENRMNT','G3FRLNCH']



student_df = pd.read_csv('https://drive.google.com/uc?id=1O-3EEu5Vf4IKuDpKXpgog4bdT4UE_o2i', usecols=students_columns)
student_df['Aide'] = student_df['g3classtype'].map(lambda x: 0 if x != "REGULAR + AIDE CLASS" else 1)


class_size_median = student_df['g3classsize'].median()
student_df['treatment'] = student_df['g3classsize'].map(lambda x: 0 if x < class_size_median else 1)
student_df = student_df.drop(columns = ['g3classtype','g3classsize'])
k_3_schools = pd.read_csv('https://drive.google.com/uc?id=1dHnYgWuf5IWMi7H3aR3ZKK1hufJkXsF5', usecols=schools_columns)
k_3_schools['attenRatio'] = k_3_schools['G3AVGDAT']/k_3_schools['G3ENRMNT']
k_3_schools = k_3_schools.drop('G3AVGDAT', axis='columns')
k_3_schools = k_3_schools.drop('G3ENRMNT', axis='columns')


In [ ]:
student_df = student_df.dropna(how='any')
student_df = student_df[student_df['g1speced']=='NO']
student_df = student_df.drop('g1speced', axis='columns')
k_3_schools = k_3_schools.dropna(how='any')
joined_df = student_df.merge(k_3_schools, left_on="g3schid", right_on="schid", how="inner")
joined_df = joined_df.drop(['g3schid', 'schid'], axis=1) 

In [ ]:
joined_df['gender'] = joined_df['gender'].map({'MALE':1, 'FEMALE':0})
joined_df['g3freelunch'] = joined_df['g3freelunch'].map({'FREE LUNCH':1, 'NON-FREE LUNCH': 0})
joined_df = pd.get_dummies(joined_df, columns=['SCHLURBN','race','g3thighdegree'])

In [ ]:
joined_df

,gender,yearssmall,g1tmathss,g3tyears,g3freelunch,g3absent,g3tmathss,Aide,treatment,G3FRLNCH,attenRatio,SCHLURBN_INNER CITY,SCHLURBN_RURAL,SCHLURBN_SUBURBAN,SCHLURBN_URBAN,race_ASIAN,race_BLACK,race_HISPANIC,race_OTHER,race_WHITE,g3thighdegree_BACHELORS,g3thighdegree_MASTERS,g3thighdegree_SPECIALIST
0,1,0,578.0,15.0,0,5.0,586.0,1,1,51.0,0.930435,0,1,0,0,0,0,0,0,1,0,1,0
1,0,0,545.0,15.0,1,9.0,604.0,1,1,51.0,0.930435,0,1,0,0,0,0,0,0,1,0,1,0
2,0,0,486.0,19.0,1,2.0,582.0,1,1,51.0,0.930435,0,1,0,0,0,0,0,0,1,1,0,0
3,1,0,542.0,19.0,0,8.0,533.0,1,1,51.0,0.930435,0,1,0,0,0,0,0,0,1,1,0,0
4,1,1,529.0,9.0,0,2.0,550.0,0,0,51.0,0.930435,0,1,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,1,4,523.0,14.0,1,7.0,631.0,0,0,95.0,0.940476,1,0,0,0,0,1,0,0,0,0,1,0
3760,1,4,545.0,14.0,1,5.0,624.0,0,0,95.0,0.940476,1,0,0,0,0,1,0,0,0,0,1,0
3761,0,0,529.0,17.0,1,7.0,573.0,1,1,95.0,0.940476,1,0,0,0,0,1,0,0,0,1,0,0
3762,0,4,523.0,14.0,1,12.0,570.0,0,0,95.0,0.940476,1,0,0,0,0,1,0,0,0,0,1,0


In [ ]:
to_scale = ['g1tmathss', 'g3tmathss']
scaler = MinMaxScaler()

for col in to_scale:
  joined_df[col] = scaler.fit_transform(joined_df[[col]])

##ATE Calculations:

In [ ]:
# Check with model is the best
X = joined_df.drop(columns= ['g3tmathss','treatment'])
X_train, X_test, y_train, y_test = train_test_split(X, joined_df["treatment"], test_size=0.33, random_state=42)
model_list = [LogisticRegression(solver='liblinear', max_iter=10000), RandomForestClassifier(max_depth=20)]
model_name = ["LogisticRegression","RandomForest"]
for ind, model in enumerate(model_list):
  model.fit(X_train, y_train)
  predictions = model.predict(X_test)
  print(model_name[ind] +" F1 score: " + str(f1_score(y_test, predictions, average='macro')))
  print(model_name[ind] +" accuracy: " + str(accuracy_score(y_test, predictions)))

LogisticRegression F1 score: 0.8696453976489118
LogisticRegression accuracy: 0.8720836685438456
RandomForest F1 score: 0.9935576160587952
RandomForest accuracy: 0.9935639581657281


In [ ]:
def calc_prop(df, label, treatment): # This function should only be called once.
  X = df.drop(columns= [label,treatment])
  rf = RandomForestClassifier(max_depth=8).fit(X, df[treatment])
  results = rf.predict_proba(X)
  return [res[1] for res in results]

joined_df['prop'] = calc_prop(joined_df, 'g3tmathss', 'treatment')

In [ ]:
joined_df.drop('prop', axis=1)

,gender,yearssmall,g1tmathss,g3tyears,g3freelunch,g3absent,g3tmathss,Aide,treatment,G3FRLNCH,attenRatio,SCHLURBN_INNER CITY,SCHLURBN_RURAL,SCHLURBN_SUBURBAN,SCHLURBN_URBAN,race_ASIAN,race_BLACK,race_HISPANIC,race_OTHER,race_WHITE,g3thighdegree_BACHELORS,g3thighdegree_MASTERS,g3thighdegree_SPECIALIST
0,1,0,0.614173,15.0,0,5.0,0.344948,1,1,51.0,0.930435,0,1,0,0,0,0,0,0,1,0,1,0
1,0,0,0.484252,15.0,1,9.0,0.407666,1,1,51.0,0.930435,0,1,0,0,0,0,0,0,1,0,1,0
2,0,0,0.251969,19.0,1,2.0,0.331010,1,1,51.0,0.930435,0,1,0,0,0,0,0,0,1,1,0,0
3,1,0,0.472441,19.0,0,8.0,0.160279,1,1,51.0,0.930435,0,1,0,0,0,0,0,0,1,1,0,0
4,1,1,0.421260,9.0,0,2.0,0.219512,0,0,51.0,0.930435,0,1,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,1,4,0.397638,14.0,1,7.0,0.501742,0,0,95.0,0.940476,1,0,0,0,0,1,0,0,0,0,1,0
3760,1,4,0.484252,14.0,1,5.0,0.477352,0,0,95.0,0.940476,1,0,0,0,0,1,0,0,0,0,1,0
3761,0,0,0.421260,17.0,1,7.0,0.299652,1,1,95.0,0.940476,1,0,0,0,0,1,0,0,0,1,0,0
3762,0,4,0.397638,14.0,1,12.0,0.289199,0,0,95.0,0.940476,1,0,0,0,0,1,0,0,0,0,1,0


In [ ]:
def S_learner(df, label, treatment):
  boot = []
  for i in range(BOOT_NUM):
    subset = df.sample(frac=0.8)
    XT = subset.drop(columns= [label])
    Y = subset[label]
    model = RandomForestRegressor().fit(XT, Y)
    XT = XT[subset.drop(columns= [label,treatment]).columns].assign(treatment=np.ones(len(XT)))
    XT_counterf = XT[subset.drop(columns= [label,treatment]).columns].assign(treatment=np.zeros(len(XT)))
    boot.append((1 / len(XT)) * (model.predict(XT) - model.predict(XT_counterf)).sum())
  return sum(boot)/len(boot), (np.quantile(boot, 0.025),  np.quantile(boot, 0.975))
ATE_S_LEARNER, CI_S_LEARNER = S_learner(joined_df.drop('prop', axis=1), 'g3tmathss', 'treatment')
print(ATE_S_LEARNER)
print(CI_S_LEARNER)

-0.002466052835857592
(-0.008234458360382819, 0.000951619902401999)


In [ ]:
def T_learner(df, label, treatment):
  boot = []
  for i in range(BOOT_NUM):
    subset = df.sample(frac=0.8)
    XT = subset.drop(columns= [label])
    Y = subset[label]
    YT = subset[[label,treatment]]
    Y_0 = YT.loc[YT[treatment] == 0]
    Y_1 = YT.loc[YT[treatment] == 1]
    X_0 = XT.loc[XT[treatment] == 0]
    X_1 = XT.loc[XT[treatment] == 1]

    model_0 = RandomForestRegressor().fit(X_0.drop(columns=[treatment]), Y_0[label])
    model_1 = RandomForestRegressor().fit(X_1.drop(columns=[treatment]), Y_1[label])

    boot.append((1 / len(XT)) * (model_1.predict(XT.drop(columns=[treatment])) - model_0.predict(XT.drop(columns=[treatment]))).sum())
  return sum(boot)/len(boot), (np.quantile(boot, 0.025),  np.quantile(boot, 0.975))
ATE_T_LEARNER, CI_T_LEARNER = T_learner(joined_df.drop('prop', axis=1), 'g3tmathss', 'treatment')
print(ATE_T_LEARNER)
print(CI_T_LEARNER)

-0.001723116824513923
(-0.005876357238284554, 0.003731164657859709)


In [ ]:
def IPW(df, label, treatment):
  boot = []
  for i in range(BOOT_NUM):
    subset = df.sample(frac=0.8)
    subset_0 = subset.loc[subset['treatment'] == 0]
    subset_1 = subset.loc[subset['treatment'] == 1]
    n = len(subset)
    IPW_ATE = sum(subset_1[label] / subset_1['prop']) - sum(subset_0[label] / (1 - subset_0['prop']))
    IPW_ATE /= n
    boot.append(IPW_ATE)
  return sum(boot)/len(boot), (np.quantile(boot, 0.025),  np.quantile(boot, 0.975))
ATE_IPW, CI_IPW = IPW(joined_df, 'g3tmathss', 'treatment')
print(ATE_IPW)
print(CI_IPW)  

-0.007826047199238672
(-0.017147928963805313, 0.0022718130973142145)


In [ ]:
def matching(df, label, treatment):
  boot = []
  for i in range(BOOT_NUM):
    subset = df.sample(frac=0.8)
    X = subset.drop(columns= [label,treatment])
    Y = subset[label]
    x_1 = X.loc[subset[treatment] == 1]
    y_1 = Y.loc[subset[treatment] == 1]
    y_0 = Y.loc[subset[treatment] == 0]
    x_0 = X.loc[subset[treatment] == 0]
    knn0 = KNeighborsRegressor(1)
    knn1 = KNeighborsRegressor(1)
    knn0.fit(x_0, y_0)
    knn1.fit(x_1, y_1)
    ite_1 = list(y_1 - knn0.predict(x_1))
    ite_0 = list(knn1.predict(x_0)-y_0)
    boot.append(np.mean(ite_0+ite_1))
  return sum(boot)/len(boot), (np.quantile(boot, 0.025),  np.quantile(boot, 0.975))
ATE_MATCHING, CI_MATCHING = matching(joined_df.drop('prop', axis=1), 'g3tmathss', 'treatment')
print(ATE_MATCHING)
print(CI_MATCHING) 

-0.013544170793038763
(-0.0184948452653858, -0.007218074956286876)


In [ ]:
# x_label = ['S_LEARNER', 'T_LEARNER', 'IPW', 'MATCHING']
# y_ATE = [ATE_S_LEARNER, ATE_T_LEARNER,ATE_IPW,ATE_MATCHING]
# y_low = [ATE_S_LEARNER-CI_S_LEARNER[0],ATE_T_LEARNER-CI_T_LEARNER[0],ATE_IPW-CI_IPW[0],ATE_MATCHING-CI_MATCHING[0]]
# y_high = [ATE_S_LEARNER+CI_S_LEARNER[0],ATE_T_LEARNER+CI_T_LEARNER[0],ATE_IPW+CI_IPW[0],ATE_MATCHING+CI_MATCHING[0]]
# y_ATE = [ATE_S_LEARNER, ATE_T_LEARNER,ATE_MATCHING]
# y_low = [ATE_S_LEARNER-CI_S_LEARNER[0],ATE_T_LEARNER-CI_T_LEARNER[0],ATE_MATCHING-CI_MATCHING[0]]
# y_high = [ATE_S_LEARNER+CI_S_LEARNER[0],ATE_T_LEARNER+CI_T_LEARNER[0],ATE_MATCHING+CI_MATCHING[0]]
color_list = ["mediumseagreen", "slateblue","deepskyblue", "midnightblue"]

x_label = ['ֹTֹ_ֹLEARNER','S_LEARNER', 'IPW', 'MATCHING']
y_ATE = [ATE_T_LEARNER, ATE_S_LEARNER, ATE_IPW,ATE_MATCHING]
y_low = [CI_T_LEARNER[0],CI_S_LEARNER[0],CI_IPW[0],CI_MATCHING[0]]
y_high = [CI_T_LEARNER[1],CI_S_LEARNER[1],CI_IPW[1],CI_MATCHING[1]]
print(y_ATE)
print(y_low)
print(y_high)
fig = go.Figure()
# first I add a trace for every x
fig.add_trace(go.Scatter(x=x_label,
                         y=y_low,
                         mode="markers",
                         showlegend=False,
                         marker_color=color_list,
                         marker=dict(size=10)))

fig.add_trace(go.Scatter(x=x_label,
                         y=y_high,
                         mode="markers",
                         showlegend=False,
                         marker_color=color_list,
                         marker=dict(size=10)))

fig.add_trace(go.Scatter(x=x_label,
                         y=y_ATE,
                         mode="markers",
                         showlegend=False,
                         marker_color=color_list,
                         marker=dict(size=10)))

for i in range(len(x_label)):
    fig.add_shape(
        dict(type="line", x0=x_label[i], x1=x_label[i],y0=y_low[i],y1=y_high[i],line=dict(color=color_list[i],width=2))
    )
fig.update_layout(title="CI", title_x=0.5)
fig.show()

[-0.001723116824513923, -0.002466052835857592, -0.007826047199238672, -0.013544170793038763]
[-0.005876357238284554, -0.008234458360382819, -0.017147928963805313, -0.0184948452653858]
[0.003731164657859709, 0.000951619902401999, 0.0022718130973142145, -0.007218074956286876]


In [ ]:
df_result = pd.DataFrame(list(zip(x_label,y_low,y_high,y_ATE)), columns=['method','lower bound CI','upper bound CI','ATE'])

In [ ]:
df_result.reset_index(drop=True)

,method,lower bound CI,upper bound CI,ATE
0,ֹTֹ_ֹLEARNER,-0.005876,0.003731,-0.001723
1,S_LEARNER,-0.008234,0.000952,-0.002466
2,IPW,-0.017148,0.002272,-0.007826
3,MATCHING,-0.018495,-0.007218,-0.013544
